# imdb电影评论情感分析

In [1]:
import numpy as np
import time
import datetime
import os
import re
from random import randint
from os.path import isfile, join
import tensorflow as tf

In [2]:
wordsList = np.load('/Users/zouhao/Desktop/LSTM情感分析Test4/wordsList.npy')
print('载入word列表:',np.shape(wordsList),type(wordsList))
# 这里转化为列表,是二进制编码

载入word列表: (400000,) <class 'numpy.ndarray'>


In [3]:
wordsList = wordsList.tolist()
# 转化为utf-8编码的形式
wordsList = [word.decode('UTF-8')  for word in wordsList]

In [4]:
wordsList#list

['0',
 ',',
 '.',
 'of',
 'to',
 'and',
 'in',
 'a',
 '"',
 "'s",
 'for',
 '-',
 'that',
 'on',
 'is',
 'was',
 'said',
 'with',
 'he',
 'as',
 'it',
 'by',
 'at',
 '(',
 ')',
 'from',
 'his',
 "''",
 '``',
 'an',
 'be',
 'has',
 'are',
 'have',
 'but',
 'were',
 'not',
 'this',
 'who',
 'they',
 'had',
 'i',
 'which',
 'will',
 'their',
 ':',
 'or',
 'its',
 'one',
 'after',
 'new',
 'been',
 'also',
 'we',
 'would',
 'two',
 'more',
 "'",
 'first',
 'about',
 'up',
 'when',
 'year',
 'there',
 'all',
 '--',
 'out',
 'she',
 'other',
 'people',
 "n't",
 'her',
 'percent',
 'than',
 'over',
 'into',
 'last',
 'some',
 'government',
 'time',
 '$',
 'you',
 'years',
 'if',
 'no',
 'world',
 'can',
 'three',
 'do',
 ';',
 'president',
 'only',
 'state',
 'million',
 'could',
 'us',
 'most',
 '_',
 'against',
 'u.s.',
 'so',
 'them',
 'what',
 'him',
 'united',
 'during',
 'before',
 'may',
 'since',
 'many',
 'while',
 'where',
 'states',
 'because',
 'now',
 'city',
 'made',
 'like',
 'b

In [5]:
# 400000x50的嵌入矩阵，训练好的词典向量模型
wordVectors = np.load('/Users/zouhao/Desktop/LSTM情感分析Test4/wordVectors.npy')
print('载入文本向量:',wordVectors.shape)

载入文本向量: (400000, 50)


In [6]:
wordVectors

array([[ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.013441 ,  0.23682  , -0.16899  , ..., -0.56657  ,  0.044691 ,
         0.30392  ],
       [ 0.15164  ,  0.30177  , -0.16763  , ..., -0.35652  ,  0.016413 ,
         0.10216  ],
       ...,
       [-0.51181  ,  0.058706 ,  1.0913   , ..., -0.25003  , -1.125    ,
         1.5863   ],
       [-0.75898  , -0.47426  ,  0.4737   , ...,  0.78954  , -0.014116 ,
         0.6448   ],
       [-0.79149  ,  0.86617  ,  0.11998  , ..., -0.29996  , -0.0063003,
         0.3954   ]], dtype=float32)

In [7]:
childrenIndex = wordsList.index('children')
wordVectors[childrenIndex]
#词向量库的词和列表词的索引是一一对应的

array([ 5.3888e-01,  5.0514e-01, -4.5316e-02, -1.3606e+00,  6.7850e-01,
        1.4193e+00, -8.5180e-01, -6.6912e-01,  4.5732e-01, -1.5827e-01,
       -3.7624e-02, -2.0835e-01,  5.4524e-01, -6.4056e-01,  1.2267e+00,
       -2.6497e-01, -4.5656e-01,  2.7901e-01,  3.8550e-01,  1.4437e-01,
        2.6343e-03,  1.7084e+00,  5.6702e-01,  6.0751e-01,  4.3173e-01,
       -1.1765e+00, -4.5223e-01, -9.8725e-01,  1.9464e-01, -6.4467e-01,
        3.1015e+00,  1.0881e+00, -1.7687e-01, -8.1104e-01,  2.8289e-01,
        6.5915e-01,  3.8012e-01, -2.7123e-01,  6.2490e-01, -5.4019e-02,
       -6.4090e-01,  1.0671e-01,  7.9348e-01,  1.0989e-01,  6.6735e-01,
       -6.5263e-02, -6.3952e-01, -1.2490e+00, -3.4498e-02,  1.8429e-01],
      dtype=float32)

In [8]:
pos_files = os.listdir('/Users/zouhao/Desktop/LSTM情感分析Test4/pos') #得到文件夹下的所有文件名称
neg_files = os.listdir('/Users/zouhao/Desktop/LSTM情感分析Test4/neg') #得到文件夹下的所有文件名称
pos_files

['4715_9.txt',
 '12390_8.txt',
 '8329_7.txt',
 '9063_8.txt',
 '3092_10.txt',
 '9865_8.txt',
 '6639_10.txt',
 '10460_10.txt',
 '10331_10.txt',
 '11606_10.txt',
 '6168_10.txt',
 '2712_10.txt',
 '3225_10.txt',
 '3574_10.txt',
 '3192_10.txt',
 '716_10.txt',
 '2612_10.txt',
 '5568_8.txt',
 '6554_7.txt',
 '1807_7.txt',
 '3474_10.txt',
 '11057_10.txt',
 '10231_10.txt',
 '11706_10.txt',
 '11167_9.txt',
 '803_10.txt',
 '5245_8.txt',
 '7935_8.txt',
 '835_8.txt',
 '6970_8.txt',
 '9533_9.txt',
 '5393_10.txt',
 '3384_8.txt',
 '6935_8.txt',
 '4342_10.txt',
 '9576_9.txt',
 '11913_10.txt',
 '5124_10.txt',
 '10961_9.txt',
 '2835_7.txt',
 '9919_9.txt',
 '9177_10.txt',
 '6762_9.txt',
 '12091_8.txt',
 '4414_9.txt',
 '9491_10.txt',
 '6115_10.txt',
 '7422_10.txt',
 '3258_10.txt',
 '6727_9.txt',
 '9327_8.txt',
 '4451_9.txt',
 '3509_10.txt',
 '2389_10.txt',
 '1936_10.txt',
 '6284_7.txt',
 '7727_9.txt',
 '1794_7.txt',
 '2902_9.txt',
 '3409_10.txt',
 '9591_10.txt',
 '2379_8.txt',
 '6655_7.txt',
 '58_9.txt',
 '1

In [9]:
num_words = []
for pf in pos_files:
    with open('/Users/zouhao/Desktop/LSTM情感分析Test4/pos/'+pf, "r", encoding = 'utf-8') as f:
        line = f.readline()
        counter = len(line.split())
        num_words.append(counter)
print('正面评价完结')

正面评价完结


In [10]:
for nf in neg_files:
    with open('/Users/zouhao/Desktop/LSTM情感分析Test4/neg/'+nf, "r", encoding='utf-8') as f:
        line = f.readline()
        counter = len(line.split())#也可用collections.Counter()来统计词频 
        num_words.append(counter)
print('负面评价完结')

负面评价完结


In [11]:
num_files = len(num_words)
print('文件总数', num_files)
print('所有的词的数量', sum(num_words))
print('平均文件词的长度', sum(num_words) / len(num_words))

文件总数 25000
所有的词的数量 5844680
平均文件词的长度 233.7872


In [12]:
max_seq_num = 250
num_dimensions = 50  # 每个单词向量的维度，这里和嵌入矩阵的每个词的维度相同

# arr:24 x 250的矩阵
def get_train_batch():
    labels = []
    arr = np.zeros([batch_size, max_seq_num])
    for i in range(batch_size):
        if (i % 2 == 0):
            num = randint(1, 11499)
            labels.append([1, 0])
        else:
            num = randint(13499, 24999)
            labels.append([0, 1])
        arr[i] = ids[num - 1:num]
    return arr, labels

# 同上
def get_test_batch():
    labels = []
    arr = np.zeros([batch_size, max_seq_num])
    for i in range(batch_size):
        num = randint(11499, 13499)
        if (num <= 12499):
            labels.append([1, 0])
        else:
            labels.append([0, 1])
        arr[i] = ids[num - 1:num]
    return arr, labels

In [13]:
batch_size = 24 # batch的尺寸
lstm_units = 64 # lstm的单元数量
num_labels = 2  # 输出的类别数
iterations = 200000 # 迭代的次数 
# 载入正负样本的词典映射
ids = np.load('/Users/zouhao/Desktop/LSTM情感分析Test4/idsMatrix.npy')
print('载入IDS:',ids.shape)
tf.reset_default_graph()
# 确定好单元的占位符：输入是24x300，输出是24x2
labels = tf.placeholder(tf.float32, [batch_size, num_labels])
input_data = tf.placeholder(tf.int32, [batch_size, max_seq_num])

# 必须先定义该变量
data = tf.Variable(
    tf.zeros([batch_size, max_seq_num, num_dimensions]), dtype=tf.float32)
# 调用tf.nn.lookup()接口获得文本向量，该函数返回batch_size个文本的3D张量，用于后续的训练
data = tf.nn.embedding_lookup(wordVectors, input_data)

# 使用tf.contrib.rnn.BasicLSTMCell细胞单元配置lstm的数量
lstmCell = tf.contrib.rnn.BasicLSTMCell(lstm_units)
# 配置dropout参数，以此避免过拟合
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)
# 最后将LSTM cell和数据输入到tf.nn.dynamic_rnn函数，功能是展开整个网络，并且构建一整个RNN模型
# 这里的value认为是最后的隐藏状态，该向量将重新确定维度，然后乘以一个权重加上偏置，最终获得lable
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstm_units, num_labels]))
bias = tf.Variable(tf.constant(0.1, shape=[num_labels]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

载入IDS: (25000, 250)


#### 1、使用tf.train.Saver.save()方法保存模型

    sess: 用于保存变量操作的会话。

    save_path: String类型，用于指定训练结果的保存路径。

    global_step: 如果提供的话，这个数字会添加到save_path后面，用于构建checkpoint文件。这个参数有助于我们区分不同训练阶段的结果。

    2、使用tf.train.Saver.restore方法价值模型

        sess: 用于加载变量操作的会话。

        save_path: 同保存模型是用到的的save_path参数。

        下面通过一个代码演示这两个函数的使用方法

    假设保存变量的时候是

checkpoint_filepath='models/train.ckpt'
saver.save(session,checkpoint_filepath)
则从文件读变量取值继续训练是
saver.restore(session,checkpoint_filepath)

#### 3、saver=tf.train.Saver()

假设保存变量的时候是
checkpoint_filepath='models/train.ckpt'
saver.save(session,checkpoint_filepath)
则从文件读变量取值继续训练是
saver.restore(session,checkpoint_filepath) 

In [14]:
# 定义正确的预测函数和正确率评估参数
correctPred = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

# 最后将标准的交叉熵损失函数定义为损失值，这里是以adam为优化函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)


sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())


tf.summary.scalar('loss',loss)
tf.summary.scalar('Accrar',accuracy)
merged=tf.summary.merge_all()
logdir='tensorboard/'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"/"
writer=tf.summary.FileWriter(logdir,sess.graph)


iterations = 10000
for i in range(iterations):
    # 下个批次的数据
    next_batch, next_batch_labels = get_train_batch()
    sess.run(optimizer,{input_data: next_batch, labels: next_batch_labels}) 

    # 每50次写入一次leadboard
    if(i%50==0):
        summary=sess.run(merged,{input_data: next_batch, labels: next_batch_labels})
        writer.add_summary(summary,i)

    if (i%1000==0):
        loss_ = sess.run(loss, {input_data: next_batch, labels: next_batch_labels})

        accuracy_=(sess.run(accuracy, {input_data: next_batch, labels: next_batch_labels})) * 100
        print("iteration:{}/{}".format(i+1, iterations),
                  "\nloss:{}".format(loss_),
                  "\naccuracy:{}".format(accuracy_))    
        print('..........')  
    # 每10000次保存一下模型
    if(i%10000==0 and i!=0):
        save_path=saver.save(sess,"/Users/zouhao/Desktop/LSTM情感分析Test4/models/pretrained_lstm.ckpt",global_step=i)
        print("saved to %s"% save_path)

writer.close()#4:55-5:40

iteration:1/10000 
loss:0.7354393005371094 
accuracy:62.5
..........
iteration:1001/10000 
loss:0.5787484645843506 
accuracy:79.16666865348816
..........
iteration:2001/10000 
loss:0.6139519810676575 
accuracy:62.5
..........
iteration:3001/10000 
loss:0.7065317630767822 
accuracy:50.0
..........
iteration:4001/10000 
loss:0.6635363698005676 
accuracy:41.66666567325592
..........
iteration:5001/10000 
loss:0.7302400469779968 
accuracy:50.0
..........
iteration:6001/10000 
loss:0.6600983738899231 
accuracy:50.0
..........
iteration:7001/10000 
loss:0.6846897006034851 
accuracy:66.66666865348816
..........
iteration:8001/10000 
loss:0.643425464630127 
accuracy:62.5
..........
iteration:9001/10000 
loss:0.6095423698425293 
accuracy:75.0
..........


In [14]:
# 定义正确的预测函数和正确率评估参数
correctPred = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

# 最后将标准的交叉熵损失函数定义为损失值，这里是以adam为优化函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)


sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())


tf.summary.scalar('loss',loss)
tf.summary.scalar('Accrar',accuracy)
merged=tf.summary.merge_all()
logdir='tensorboard/'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"/"
writer=tf.summary.FileWriter(logdir,sess.graph)


iterations = 1000
for i in range(iterations):
    # 下个批次的数据
    next_batch, next_batch_labels = get_train_batch()
    sess.run(optimizer,{input_data: next_batch, labels: next_batch_labels}) 

    # 每50次写入一次leadboard
    if(i%50==0):
        summary=sess.run(merged,{input_data: next_batch, labels: next_batch_labels})
        writer.add_summary(summary,i)

    if (i%100==0):
        loss_ = sess.run(loss, {input_data: next_batch, labels: next_batch_labels})

        accuracy_=(sess.run(accuracy, {input_data: next_batch, labels: next_batch_labels})) * 100
        print("iteration:{}/{}".format(i+1, iterations),
                  "\nloss:{}".format(loss_),
                  "\naccuracy:{}".format(accuracy_))    
        print('..........')  
    # 每1000次保存一下模型
    if(i%1000==0 and i!=0):
        save_path=saver.save(sess,"/Users/zouhao/Desktop/LSTM情感分析Test4/models/pretrained_lstm.ckpt",global_step=i)
        print("saved to %s"% save_path)

writer.close()

iteration:1/1000 
loss:0.835259735584259 
accuracy:54.16666865348816
..........
iteration:101/1000 
loss:0.69843989610672 
accuracy:50.0
..........
iteration:201/1000 
loss:0.6276010870933533 
accuracy:62.5
..........
iteration:301/1000 
loss:0.7385787963867188 
accuracy:66.66666865348816
..........
iteration:401/1000 
loss:0.623770534992218 
accuracy:54.16666865348816
..........
iteration:501/1000 
loss:0.7822547554969788 
accuracy:54.16666865348816
..........
iteration:601/1000 
loss:0.7419171333312988 
accuracy:54.16666865348816
..........
iteration:701/1000 
loss:0.6957378387451172 
accuracy:66.66666865348816
..........
iteration:801/1000 
loss:0.6850967407226562 
accuracy:33.33333432674408
..........
iteration:901/1000 
loss:0.632957398891449 
accuracy:62.5
..........


In [14]:
# 定义正确的预测函数和正确率评估参数
correctPred = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

# 最后将标准的交叉熵损失函数定义为损失值，这里是以adam为优化函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)


sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())


tf.summary.scalar('loss',loss)
tf.summary.scalar('Accrar',accuracy)
merged=tf.summary.merge_all()
logdir='tensorboard/'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"/"
writer=tf.summary.FileWriter(logdir,sess.graph)


iterations = 100000
for i in range(iterations):
    # 下个批次的数据
    next_batch, next_batch_labels = get_train_batch()
    sess.run(optimizer,{input_data: next_batch, labels: next_batch_labels}) 

    # 每50次写入一次leadboard
    if(i%50==0):
        summary=sess.run(merged,{input_data: next_batch, labels: next_batch_labels})
        writer.add_summary(summary,i)

    if (i%1000==0):
        loss_ = sess.run(loss, {input_data: next_batch, labels: next_batch_labels})

        accuracy_=(sess.run(accuracy, {input_data: next_batch, labels: next_batch_labels})) * 100
        print("iteration:{}/{}".format(i+1, iterations),
                  "\nloss:{}".format(loss_),
                  "\naccuracy:{}".format(accuracy_))    
        print('..........')  
    # 每10000次保存一下模型
    if(i%10000==0 and i!=0):
        save_path=saver.save(sess,"/Users/zouhao/Desktop/LSTM情感分析Test4/models/pretrained_lstm.ckpt",global_step=i)
        print("saved to %s"% save_path)

writer.close()#11:20

iteration:1/100000 
loss:0.8111233711242676 
accuracy:54.16666865348816
..........
iteration:1001/100000 
loss:0.6626638770103455 
accuracy:58.33333134651184
..........
iteration:2001/100000 
loss:0.7344972491264343 
accuracy:45.83333432674408
..........
iteration:3001/100000 
loss:0.7305406928062439 
accuracy:54.16666865348816
..........
iteration:4001/100000 
loss:0.6955063939094543 
accuracy:50.0
..........
iteration:5001/100000 
loss:0.5072590708732605 
accuracy:83.33333134651184
..........
iteration:6001/100000 
loss:0.5338289141654968 
accuracy:79.16666865348816
..........
iteration:7001/100000 
loss:0.3405703604221344 
accuracy:79.16666865348816
..........
iteration:8001/100000 
loss:0.48696961998939514 
accuracy:75.0
..........
iteration:9001/100000 
loss:0.3967016041278839 
accuracy:83.33333134651184
..........
iteration:10001/100000 
loss:0.4368619918823242 
accuracy:83.33333134651184
..........
saved to /Users/zouhao/Desktop/LSTM情感分析Test4/models/pretrained_lstm.ckpt-10000
it

iteration:94001/100000 
loss:0.023729274049401283 
accuracy:95.83333134651184
..........
iteration:95001/100000 
loss:0.003417074680328369 
accuracy:100.0
..........
iteration:96001/100000 
loss:0.01309977937489748 
accuracy:100.0
..........
iteration:97001/100000 
loss:0.011841499246656895 
accuracy:100.0
..........
iteration:98001/100000 
loss:0.001950435689650476 
accuracy:100.0
..........
iteration:99001/100000 
loss:0.06551812589168549 
accuracy:100.0
..........


In [ ]:
#11:20 11:44 4:00

### 用模型进行预测

In [1]:
path="/Users/zouhao/Desktop/0_10.txt"
with open(path,'r') as f:
    line = f.readline().strip() 

In [4]:
line

"I went and saw this movie last night after being coaxed to by a few friends of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher played the character of Jake Fischer very well, and Kevin Costner played Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which was sold out) was overcome by laughter during the first half of the movie, and were moved to tears during the second half. While exiting the theater I not only saw many women in tears, but many full grown men as well, trying desperately not to let anyone see them crying. This movie was great, and I suggest that you go see it before you judge."

In [26]:
sess=tf.Session()    
saver = tf.train.import_meta_graph('/Users/zouhao/Desktop/LSTM情感分析Test4/models/pretrained_lstm.ckpt-90000.meta')
saver.restore(sess,tf.train.latest_checkpoint('/Users/zouhao/Desktop/LSTM情感分析Test4/models/'))
 
graph = tf.get_default_graph()
w1 = graph.get_tensor_by_name("w1:0")
w2 = graph.get_tensor_by_name("w2:0")
feed_dict ={w1:13.0,w2:17.0}
 
op_to_restore = graph.get_tensor_by_name("op_to_restore:0")
 
print(sess.run(op_to_restore,feed_dict))
sess.close()  

INFO:tensorflow:Restoring parameters from /Users/zouhao/Desktop/LSTM情感分析Test4/models/pretrained_lstm.ckpt-90000


KeyError: "The name 'w1:0' refers to a Tensor which does not exist. The operation, 'w1', does not exist in the graph."